In [19]:
import pandas as pd
from tqdm import tqdm
import networkx as nx
from networkx import from_pandas_dataframe

ImportError: cannot import name 'from_pandas_dataframe' from 'networkx' (/home/simone/anaconda3/lib/python3.9/site-packages/networkx/__init__.py)

In [5]:
a_q=pd.read_csv('a2q',sep=' ', names=['u','v','t'])  #answers to questions
c_a=pd.read_csv('c2a',sep=' ', names=['u','v','t'])  #comments to answers
c_q=pd.read_csv('c2q',sep=' ', names=['u','v','t'])  #comments to questions

In [6]:
print(a_q.shape,c_a.shape,c_q.shape)

(17823525, 3) (25405374, 3) (20268151, 3)


In [7]:
a_q=a_q.sort_values('t')
c_a=c_a.sort_values('t')
c_q=c_q.sort_values('t')

#change time to hours
a_q['t']=a_q['t'].div(3600)
c_a['t']=c_a['t'].div(3600)
c_q['t']=c_q['t'].div(3600)

#add type column to dataframes
a_q['type']='a_q'
c_a['type']='c_a'
c_q['type']='c_q'

In [16]:
def time_maxmin(data,min,max): #Retrieve maximum and minimum timestamps
    m=data['t'].min()
    M=data['t'].max()
    if m<min:
        min=m
    if M>max:
        max=M
    return min,max

In [17]:
m=10**(10)
M=0
m, M = time_maxmin(a_q, m, M)
m, M = time_maxmin(c_a, m, M)
m, M = time_maxmin(c_q, m, M)
print(m, M)

338213.2991666667 404798.1744444444


In [21]:
def create_graph(data1,data2,data3,time_window,self_loops=0):
    if self_loops==0: #remove rows with u=v
        data1 = data1[data1['u'] != data1['v']]
        data2 = data2[data2['u'] != data2['v']]
        data3 = data3[data3['u'] != data3['v']]
    
    #remove rows not having time in given time interval
    data1 = data1[data1['t'].between(time_window[0], time_window[1])]
    data2 = data2[data2['t'].between(time_window[0], time_window[1])]
    data3 = data3[data3['t'].between(time_window[0], time_window[1])]

    
    A= nx.MultiDiGraph()
    B = nx.MultiDiGraph()
    C = nx.MultiDiGraph()
    A=nx.from_pandas_dataframe(data1, 'u', 'v', ['t', 'type'])
    B=nx.from_pandas_dataframe(data2, 'u', 'v', ['t', 'type'])
    C=nx.from_pandas_dataframe(data3, 'u', 'v', ['t', 'type'])
    G = nx.compose(A,B)
    G = nx.compose(G,C)
    return G

In [22]:
time_window = (M/2,M/2+1000)
G=create_graph(a_q,c_a,c_q,time_window)

AttributeError: module networkx has no attribute from_pandas_dataframe